# Convolutional Layer Output Filter Count Comparison

In standard VGG-Net architectures, one general design guideline is that
consecutive convolutional layers have the same number of filters, while
for convolutional layers that follow a max pooling layer,

- No. filters = Prev. pool. layer's window size * prev. conv. layer's 
no. filters  

In this notebook, we shall continue to follow these guidelines wherever
they are observed in the baseline architecture we selected. Thus, we
shall only compare between different values of output filter counts for
the first convolutional layer, given its ripple effect on the rest of 
the model.

In the standard VGG-13 architecture, the starting output filter count is
64. We shall compare between the powers of 2, from 16 to 64.


In [ ]:
from gc import collect
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
from h5py import File
from pandas import read_csv
from sklearn.metrics import accuracy_score, classification_report, \
    confusion_matrix
from tensorflow.config.experimental import list_physical_devices, \
    set_memory_growth
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import CSVLogger, LearningRateScheduler, \
    ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, MaxPool2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.random import set_random_seed


### Model Creation Function


In [ ]:
# VGG-13
def create_model(input_shape: Tuple[int, int, int], num_classes: int,
                 num_start_filters: int = 64) -> Model:
    inputs = Input(shape=input_shape)
    
    layer = Conv2D(filters=num_start_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal", 
                   bias_initializer="he_normal")(inputs)
    layer = Conv2D(filters=num_start_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal", 
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    new_num_filters = num_start_filters * 2

    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    new_num_filters = new_num_filters * 2

    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    new_num_filters = new_num_filters * 2

    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = Conv2D(filters=new_num_filters, kernel_size=(3, 3), strides=1, 
                   padding="same", activation="relu", 
                   kernel_initializer="he_normal",
                   bias_initializer="he_normal")(layer)
    layer = MaxPool2D(pool_size=(2, 2), strides=2)(layer)
    
    layer = Flatten()(layer)
    layer = Dense(units=4096, activation="relu", 
                  kernel_initializer="he_normal",
                  bias_initializer="he_normal")(layer)
    layer = Dense(units=4096, activation="relu", 
                  kernel_initializer="he_normal",
                  bias_initializer="he_normal")(layer)
    layer = Dense(num_classes, activation="softmax")(layer)
    
    model = Model(inputs=inputs, outputs=layer)
    model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss="categorical_crossentropy", metrics=["accuracy"])
    
    return model


### Other Functions


In [ ]:
def refresh_session():
    # Call this before training a new model, to free up memory from the 
    # previous model
    clear_session()
    try:
        del model
    except NameError:
        pass
    collect()
    
    
def import_dataset(filepath: str = "./dataset.hdf5") \
        -> Tuple[np.ndarray, np.ndarray, np.ndarray, 
                 np.ndarray, np.ndarray, np.ndarray]:
    file = File(filepath, "r")
    train_data = file.get("tr_data")[()]
    val_data = file.get("val_data")[()]
    test_data = file.get("ts_data")[()]
    train_labels = file.get("tr_labels")[()]
    val_labels = file.get("val_labels")[()]
    test_labels = file.get("ts_labels")[()]
    
    return train_data, val_data, test_data, \
           train_labels, val_labels, test_labels


def get_test_results(test_model: Model, test_data: np.ndarray, 
                     test_labels: np.ndarray) -> Tuple:
    predicts = test_model.predict(test_data)
    pred_out = np.argmax(predicts, axis=1)
    test_out = np.argmax(test_labels, axis=1)
    labels = ["car", "heavy vehicles", "motorcycle"]
    
    return accuracy_score(test_out, pred_out), \
           confusion_matrix(test_out, pred_out), \
           classification_report(test_out, pred_out, target_names=labels)


def get_learn_rate(epoch: int) -> float:
    if epoch <= 10:
        lr = 1e-4
    elif epoch <= 20:
        lr = 5e-5
    elif epoch <= 30:
        lr = 1e-5
    elif epoch <= 40:
        lr = 5e-6
    else:
        lr = 1e-6
    print(f"Learning rate: {lr}")
    
    return lr


### Initialise Environment

In [ ]:
# Configure tensorflow to optimise GPU utilisation
gpu_list = list_physical_devices("GPU")
for gpu in gpu_list:
    set_memory_growth(gpu, True)
del gpu_list

# Fix tensorflow random seed
set_random_seed(324)

tr_dat, val_dat, ts_dat, tr_lbls, val_lbls, ts_lbls = import_dataset()

in_shape = (tr_dat.shape[1], tr_dat.shape[2], tr_dat.shape[3])
num_cls = tr_lbls.shape[1]

lr_scheduler = LearningRateScheduler(get_learn_rate)
early_stopper = EarlyStopping(monitor="val_loss", patience=10, verbose=1, 
                              mode="min", restore_best_weights=True) 

# Test data is not needed in this notebook, so free up the memory
del ts_dat
del ts_lbls
collect()


### Model Creation and Training

In [ ]:
# 16 starting filters
refresh_session()
data_gen = ImageDataGenerator(
    rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, horizontal_flip=True)

start_filters = 16
model = create_model(in_shape, num_cls, num_start_filters=start_filters)
print(model.summary())

checkpoint = ModelCheckpoint("./trained_models/16filters_best.hdf5", 
                             monitor="val_loss", verbose=0, 
                             save_best_only=True, mode="min")
logger = CSVLogger("./training_logs/16filters_log.csv")

model.fit_generator(
    data_gen.flow(tr_dat, tr_lbls, batch_size=32, shuffle=True), 
    steps_per_epoch=(len(tr_dat) / 32), epochs=50, verbose=2, 
    callbacks=[checkpoint, logger, lr_scheduler, early_stopper], 
    validation_data=(val_dat, val_lbls))


In [ ]:
# 32 starting filters
refresh_session()
data_gen = ImageDataGenerator(
    rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    zoom_range=0.2, horizontal_flip=True)

start_filters = 32
model = create_model(in_shape, num_cls, num_start_filters=start_filters)
print(model.summary())

checkpoint = ModelCheckpoint("./trained_models/32filters_best.hdf5", 
                             monitor="val_loss", verbose=0, 
                             save_best_only=True, mode="min")
logger = CSVLogger("./training_logs/32filters_log.csv")

model.fit_generator(
    data_gen.flow(tr_dat, tr_lbls, batch_size=32, shuffle=True), 
    steps_per_epoch=(len(tr_dat) / 32), epochs=50, verbose=2, 
    callbacks=[checkpoint, logger, lr_scheduler, early_stopper], 
    validation_data=(val_dat, val_lbls))


In [ ]:
# Training phase is complete: free training data memory
del tr_dat
del tr_lbls
refresh_session()


### Data Visualisation

In [ ]:
acc_scores = dict()
conf_matrices = dict()
class_reports = dict()

plt.style.use("ggplot")


In [ ]:
# 16 starting filters
refresh_session()
model = load_model("./trained_models/16filters_best.hdf5")
acc_scores[16], conf_matrices[16], class_reports[16] \
    = get_test_results(model, val_dat, val_lbls)

print(f"Validation accuracy for 16 starting output filters: {acc_scores[16]}")
print("Confusion Matrix:")
print(conf_matrices[16])
print(class_reports[16])

log_filter16 = read_csv("./training_logs/16filters_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_filter16["val_loss"], label="loss")
plt.plot(log_filter16["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_filter16["loss"], label="train loss")
plt.plot(log_filter16["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_filter16["acc"], label="train accuracy")
plt.plot(log_filter16["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))


In [ ]:
# 32 starting filters
refresh_session()
model = load_model("./trained_models/32filters_best.hdf5")
acc_scores[32], conf_matrices[32], class_reports[32] \
    = get_test_results(model, val_dat, val_lbls)

print(f"Validation accuracy for 32 starting output filters: {acc_scores[32]}")
print("Confusion Matrix:")
print(conf_matrices[32])
print(class_reports[32])

log_filter32 = read_csv("./training_logs/32filters_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_filter32["val_loss"], label="loss")
plt.plot(log_filter32["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_filter32["loss"], label="train loss")
plt.plot(log_filter32["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_filter32["acc"], label="train accuracy")
plt.plot(log_filter32["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))



In [ ]:
# 64 starting filters - baseline
refresh_session()
model = load_model("./trained_models/vgg13_best.hdf5")
acc_scores[64], conf_matrices[64], class_reports[64] \
    = get_test_results(model, val_dat, val_lbls)

print(f"Validation accuracy for 64 starting output filters: {acc_scores[64]}")
print("Confusion Matrix:")
print(conf_matrices[64])
print(class_reports[64])

log_filter64 = read_csv("./training_logs/vgg13_log.csv")

plt.figure(figsize=[12.5, 12.5])
plt.subplot(311)
plt.plot(log_filter64["val_loss"], label="loss")
plt.plot(log_filter64["val_acc"], label="accuracy")
plt.title("Validation")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(312)
plt.plot(log_filter64["loss"], label="train loss")
plt.plot(log_filter64["val_loss"], label="validation loss")
plt.title("Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(313)
plt.plot(log_filter64["acc"], label="train accuracy")
plt.plot(log_filter64["val_acc"], label="validation accuracy")
plt.title("Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))


#### Cross-Model Comparison

In [ ]:
plt.figure(figsize=[10, 7.5])
plt.subplot(211)
plt.plot(log_filter16["val_loss"], label="16 filters")
plt.plot(log_filter32["val_loss"], label="32 filters")
plt.plot(log_filter64["val_loss"], label="64 filters")
plt.title("Validation Loss")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))

plt.subplot(212)
plt.plot(log_filter16["val_acc"], label="16 filters")
plt.plot(log_filter32["val_acc"], label="32 filters")
plt.plot(log_filter64["val_acc"], label="64 filters")
plt.title("Validation Accuracy")
plt.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0))



